In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy

import tensorboard

from loss import Cross_Entropy_Loss
from model import Residual_CNN

#데이터셋 구축

#CREloss function 

In [1]:
import torch
import torch.nn as nn
import numpy as np
output = [0.8982, 0.805, 0.6393, 0.9983, 0.5731, 0.0469, 0.556, 0.1476, 0.8404, 0.5544]
target = [1]
loss = np.log(sum(np.exp(output))) - output[target[0]]
print(loss) # 2.143818427948945

2.143818427948945


In [25]:
output = torch.Tensor([[0.8982, 0.805, 0.6393, 0.9983, 0.5731, 0.0469, 0.556, 0.1476, 0.8404, 0.5544]])
target = torch.LongTensor([1])
criterion = nn.CrossEntropyLoss()
loss = criterion(output, target)
print(loss) # tensor(2.1438)

tensor(2.1438)


In [31]:
output = [0.8982, 0.805, 0.6393, 0.9983, 0.5731, 0.0469, 0.556, 0.1476, 0.8404, 0.5544]
target = [0,1,0,0,0,0,0,0,0,0]
print(len(target))
loss=0
# loss =  np.exp(output))
loss=np.sum(np.log(np.exp(output) / np.sum(np.exp(output)))*target)
print(-1*loss)

10
2.143818427948945
